In [2]:

pip install cv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 38.8/38.8 MB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import cv2
# Define image paths (replace with your actual paths)
cancer_path = "./CANCER"
healthy_path = "./HEALTHY"

# Function to load and preprocess images
def load_and_preprocess_images(img_path):
    images = []
    labels = []
    for filename in os.listdir(img_path):
        img = cv2.imread(os.path.join(img_path, filename), cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:  # Handle potential loading errors
            img = cv2.resize(img, (128, 128))  # Resize to consistent size
            images.append(img.flatten())
            labels.append(1 if "CANCER" in img_path else 0)  # Label based on folder name
    return np.array(images), np.array(labels)

# Load and preprocess images
cancer_images, cancer_labels = load_and_preprocess_images(cancer_path)
healthy_images, healthy_labels = load_and_preprocess_images(healthy_path)

# Combine data and labels
images = np.concatenate((cancer_images, healthy_images))
labels = np.concatenate((cancer_labels, healthy_labels))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)  # Set random state for reproducibility

# Standardize features (optional, but often recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust n_estimators as needed
rf_model.fit(X_train, y_train)

# Train XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)  # Adjust n_estimators as needed
xgb_model.fit(X_train, y_train)

# Make predictions on test set
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

# Evaluate performance using accuracy and classification report
print("Random Forest:")
print(f"Accuracy: {accuracy_score(y_test, rf_predictions)}")
print(classification_report(y_test, rf_predictions))

print("\nXGBoost:")
print(f"Accuracy: {accuracy_score(y_test, xgb_predictions)}")
print(classification_report(y_test, xgb_predictions))


Random Forest:
Accuracy: 0.8823529411764706
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        23
           1       0.91      0.91      0.91        45

    accuracy                           0.88        68
   macro avg       0.87      0.87      0.87        68
weighted avg       0.88      0.88      0.88        68


XGBoost:
Accuracy: 0.9264705882352942
              precision    recall  f1-score   support

           0       0.88      0.91      0.89        23
           1       0.95      0.93      0.94        45

    accuracy                           0.93        68
   macro avg       0.91      0.92      0.92        68
weighted avg       0.93      0.93      0.93        68



In [ ]:
models_ensemble = {}